In [6]:
import pandas as pd
import numpy as np
import re

In [2]:
re.split('[\xa0 ]', '\xa0его: \n\n—\xa0А\xa0как твоё имя,')

['', 'его:', '\n\n—', 'А', 'как', 'твоё', 'имя,']

In [3]:
re.sub('[\n]+', "\n", "\n\n\n")

'\n'

In [11]:
df = pd.read_csv('../pb-stylistic/free.csv')

In [12]:
df['len'] = df['text'].astype(str).apply(lambda x: len(x))

In [57]:
df['word_per_line'] = df['text'].astype(str).apply(
    lambda x: np.mean(
        [len(re.split('[\xa0 ]', l)) for l in re.sub('[\n]+', '\n', x).split('\n')]
    )
)

In [58]:
df.describe()

,len,char_per_line,word_per_line
count,49041.000000,49041.000000,49041.000000
mean,25099.804776,102.224800,31.117630
std,30922.766593,224.353268,45.813765
min,2.000000,2.000000,1.000000
25%,8990.000000,39.645070,13.651961
50%,24130.000000,77.777778,25.181818
75%,30273.000000,121.153846,37.823529
max,924753.000000,10497.000000,1919.500000


In [63]:
df

,link,title,text,len,char_per_line,word_per_line
0,https://ridero.ru/books/rasskazy_iz_malenkoi_k...,Рассказы из маленькой комнаты,Книга первая. Рахат-лукум \n\nВ пустынной мест...,25543,107.697872,35.500000
1,https://ridero.ru/books/tolstyi_i_slepoi,Толстый и слепой,Пролог \n\nЗдравствуйте дорогие читатели. В ва...,21325,113.655914,37.892473
2,https://ridero.ru/books/dorogi_vedushie_v_ad,"Дороги, ведущие в ад","Съежившись под колючим одеялом, можно было уже...",16218,112.419580,35.625000
3,https://ridero.ru/books/rossiya.putin.2018,Россия. Путин. 2018,"От автора \n\nНачать, пожалуй, стоит с того, ч...",29152,228.551181,63.765625
4,https://ridero.ru/books/osushestvlenie_ozhidae...,Осуществление ожидаемого,Памяти всех невинно убиенных и \n\nпострадавши...,30130,208.243056,57.422535
5,https://ridero.ru/books/ierusalim_i_ego_obitateli,Иерусалим и его обитатели,От автора \n\nАвтор не ставит перед собой цель...,20053,205.742268,57.591837
6,https://ridero.ru/books/terror_v_nebe,Террор в небе,«Террор в небе» — это книга не о пропаганде те...,30843,53.398589,18.225352
7,https://ridero.ru/books/moya_struna,Моя струна,Сказки маленьких городов \n\nС той стороны зер...,22350,78.540925,26.557143
8,https://ridero.ru/books/bata,Бата,Предисловие \n\nКаждая эпоха имеет свои особен...,29655,162.845304,51.153846
9,https://ridero.ru/books/moi_esse_stikhi_i_mysl...,Цитаты и стихи о жизни и любви,О книге Знания — это путь к просветлению \n\nЭ...,22267,114.378238,35.257732


In [7]:
import requests
import re

In [8]:
from tqdm.auto import tqdm

In [99]:
data = {}
genres = set()

In [16]:
from multiprocessing import Pool as ThreadPool

def process(data):
    i, link, data, genres = data
    if i in data.keys():
        return
    r = requests.get(link)
    if r.status_code != 200:
        print(r.status_code, link)
        return
    soup = BeautifulSoup(r.text)
    g = [d.text for d in soup.find_all('a', {'class': lambda x: x and x.startswith('index__label___')})]
    genres.update(g)
    data[i] = {
        'link': link,
        'genres': g
    }
    try:
        rating = float(soup.find('div', {'class': lambda x: x and x.startswith('index__ratingStar')}).contents[-1].text)
    except:
        print('cant get rating of ', link)
        return
    data[i]['rating'] = rating
    time.sleep(1)

alldata = list((i, link, data, genres) for i, link in enumerate(df['link'][:3000]))


pool = ThreadPool(3)

results = pool.map(process, alldata)

pool.close()
pool.join()

404 https://ridero.ru/books/nositeli
404 https://ridero.ru/books/vozmezdie_vysshikh_sil
404 https://ridero.ru/books/vongozero
404 https://ridero.ru/books/prodazhi_na_vystupleniyakh_s_nulya_do_milliona
404 https://ridero.ru/books/faktura
404 https://ridero.ru/books/mat_kakoi_byt_nelzya
404 https://ridero.ru/books/vozdayanie_monstru
527
404 https://ridero.ru/books/obychnyi_den_nesmotrya_na_monstrov
404 https://ridero.ru/books/kris_kristina
404 https://ridero.ru/books/koshki_ne_vsegda_molchat
343
404 https://ridero.ru/books/ne_prosto_o_lyubvi
404 https://ridero.ru/books/adychanskaya_tragediya
404 https://ridero.ru/books/cvetochnyi_po-drugomu
815
619
433
404 https://ridero.ru/books/kak_vylechit_rak
404 https://ridero.ru/books/khrustalnye_zvyozdy
404 https://ridero.ru/books/10_sekretov_uverennosti_v_sebe
621
404 https://ridero.ru/books/koldunya_v_dele
438
404 https://ridero.ru/books/tyomnykh_del_mastera
404 https://ridero.ru/books/mezhdu_nami_lyod
994
404 https://ridero.ru/books/rozhdestven

2343
1919
2107
2344
1920
2108
cant get rating of  https://ridero.ru/books/zdorovyi_obraz_zhizni
1921
2109
2346
1922
2110
1923
2347
2111
2348
1924
2112
1925
2349
2113
1926
2350
2114
1927
2351
2115
1928
2352
2116
1929
2353
2117
2354
1930
2118
2355
1931
2119
2356
1932
2357
2120
1933
2358
2121
1934
2359
2122
1935
2360
2123
2361
1936
2124
2362
1937
1938
2125
2363
cant get rating of  https://ridero.ru/books/spisok_mecht
2126
cant get rating of  https://ridero.ru/books/seryozha-pastushok
2127
1940
2365
2128
1941
2366
1942
2367
2129
1943
2130
2368
1944
2131
2369
1945
2370
2132
1946
2371
2133
1947
cant get rating of  https://ridero.ru/books/ekonomika_na_osnove_ozhidaniya_pozitiva
1948
2134
1949
2373
2135
2136
2374
1950
2137
1951
2375
2138
2139
2376
1952
1953
2140
2377
1954
404 https://ridero.ru/books/lyubov_kak_mnogo_v_etom_slove_1
2378
2141
1956
2379
2142
1957
2380
2143
1958
2381
2144
1959
cant get rating of  https://ridero.ru/books/krasnaya_roza
2383
2145
1960
2384
1961
2146
2385
1962
2147
23

In [18]:
genres

{'#МишнНонФикшн',
 'μ\xa0CEPHEI ',
 'Актуальная проза',
 'Альманах «Мю Цефея» рекомендует',
 'Альтернативная история',
 'Астрономия',
 'Аудиокниги ',
 'Бестселлеры Ridero',
 'Бизнес и экономика',
 'Бизнес-книги',
 'Биографии и мемуары',
 'Биологические науки',
 'Блокчейн и криптовалюты',
 'Вся наша жизнь — игра!',
 'Гуманитарный нонфикшн',
 'Детектив',
 'Детская литература',
 'Детская образовательная литература',
 'Дом и домоводство',
 'Драма',
 'Естественные науки',
 'Занимательное языкознание',
 'Здоровый образ жизни',
 'Здоровье',
 'Зимняя подборка',
 'Зоология',
 'Игры',
 'Информационные технологии',
 'Искусство',
 'Историческая литература',
 'Историческая проза',
 'История',
 'Киберпанк',
 'Классическая проза',
 'Книги быстрого приготовления от магистров ВШЭ',
 'Книги для писателей',
 'Книги для родителей',
 'Книги о котах и кошках',
 'Книги о собаках',
 'Книги участников  Красноярской ярмарки книжной культуры 2019',
 'Книги участников  фестиваля «Читай, Екатеринбург!» ',
 'Книги 

In [21]:
FICTION_TAGS = {
    'Актуальная проза',
    'Классическая проза',
    'Современная проза',
    'Детектив',
    'Драма',
    'Киберпанк',
    'Любовный роман',
    'Мифы и легенды',
    'Научная фантастика',
    'Сказки и сказочные истории',
    'Триллер',
    'Триллеры и детективы',
    'Ужасы и мистика',
    'Фантастика',
    'Фэнтези',
    'Юмор'
}
POETRY_TAGS = {
  'Поэзия'
}
def check_tags(tags):
    return set(tags).intersection(FICTION_TAGS) and not set(tags).intersection(POETRY_TAGS)

In [26]:
valid_books = {k:v for k, v in data.items() if check_tags(v['genres'])}

In [28]:
valid_books

{0: {'link': 'https://ridero.ru/books/rasskazy_iz_malenkoi_komnaty',
  'genres': ['Современная проза',
   'Электронная',
   'Печатная А5',
   'Русский',
   'птицы'],
  'rating': 4.7},
 2: {'link': 'https://ridero.ru/books/dorogi_vedushie_v_ad',
  'genres': ['Драма', 'Электронная', 'Печатная А5', 'Русский'],
  'rating': 4.8},
 3: {'link': 'https://ridero.ru/books/rossiya.putin.2018',
  'genres': ['Политика',
   'Публицистика',
   'Современная проза',
   'Электронная',
   'Печатная А5',
   'Русский',
   'Россия',
   'Санкт-Петербург',
   'Путин'],
  'rating': 5.0},
 6: {'link': 'https://ridero.ru/books/terror_v_nebe',
  'genres': ['Современная проза',
   'Драма',
   'Электронная',
   'Печатная А5',
   'Русский'],
  'rating': 5.0},
 7: {'link': 'https://ridero.ru/books/moya_struna',
  'genres': ['Современная проза',
   'Электронная',
   'Печатная А5',
   'Русский',
   'Новые релизы'],
  'rating': 5.0},
 8: {'link': 'https://ridero.ru/books/bata',
  'genres': ['Современная проза', 'Электро

In [40]:
catalog_url = 'https://ridero.ru/books/catalog/fiction-general-literary/russian/?sort=relevant&offset='
base_nose = 'https://ridero.store/text-utils/v1/free-text/slug/'
base_tail= '?template=domFragment&volumeLimit=0.25'
pattern_href = re.compile("/books/*")
pattern_class = re.compile("searchResult")

In [15]:
import time
import os
from bs4 import BeautifulSoup

dir_ridero = 'ridero_books'

In [30]:
r = requests.get(catalog_url+'0')

In [44]:
soup = BeautifulSoup(r.text)
soup.find_all('a', {'href': pattern_href, 'class': pattern_class})[0].get('href')

'/books/uvidimsya_v_sen_trope/'

In [48]:
from bs4 import BeautifulSoup

if not os.path.exists(dir_ridero):
    os.makedirs(dir_ridero)
df = []

for i in tqdm(range(80)):
    offset = str(i*25)
    print(offset)
    r = requests.get(catalog_url+offset)
    assert r.status_code == 200
    soup = BeautifulSoup(r.text)
    for q in soup.find_all('a', {'href': pattern_href, 'class': pattern_class}):
        id_ = q.get('href')[7:-1]
#         if id_ in os.listdir(dir_ridero):
#             continue
        author = q.find('div', {'class' : re.compile('index__authorName')}).getText()
        title = q.find('div', {'class' : re.compile('index__title')}).getText()
        rating = None
        try:
            rating = float(q.find('span', {'class': re.compile("ratingStar")}).text)
        except:
            continue
        print(author, title, id_, rating)
        content_url = base_nose + id_ + base_tail
        request = requests.get(content_url)
        if request.status_code != 200:
            print(request.status_code, content_url)
            continue
        with open(os.path.join(dir_ridero, id_), 'w') as f:
            f.write(BeautifulSoup(request.text).text)
        with open(os.path.join(dir_ridero, 'ridero_catalog_new'), 'a') as f:
            f.write(';'.join([id_, author, title, str(rating)]))
            f.write('\n')
        df.append([id_, author, title, rating])
    time.sleep(10)

0
Нигина Айви Увидимся в Сен-Тропе uvidimsya_v_sen_trope 5.0
Мара Ви Немного не в себе nemnogo_ne_v_sebe 5.0
Ирина Самохина Выйти из поезда vyiti_iz_poezda 5.0
Лев Виленский Иерусалим и его обитатели ierusalim_i_ego_obitateli 5.0
Екатерина Аполлонова Ветер между строк veter_mezhdu_strok 5.0
Илона Седдик Кахокия kakhokiya 5.0
Luciella Шестое измерение shestoe_izmerenie 4.4
Виктор Рогожкин В поисках разума v_poiskakh_razuma 5.0
Наташа Washa История Вероники Вериной istoriya_veroniki_verinoi 5.0
Александра Барвицкая Поезд 10—11 poezd_10_11 5.0
Н. Е. Пушкина Stories stories 5.0
Умберто Лопес Привет, Удача! privet_udacha 4.5
Дана Гельдэ И малое станет большим, и большое — малым i_maloe_stanet_bolshim_i_bolshoe_malym 5.0
Эндрю Огнёфф Хроники Древних Малефистериум khroniki_drevnikh_malefisterium 4.3
Ксения Блажиевская Пробуждение probuzhdenie_7 4.7
Ирина Витальевна Романенко Мастер Игры master_igry 4.9
Игорь Англер Хроники весёлой пекарни khroniki_vesyoloi_pekarni 4.8
Анастасия Лопатина Перек

Ольга Лисенкова Жили-небыли zhili-nebyli 5.0
Павел Светлый Изначальная любовь iznachalnaya_lyubov 5.0
Милла Генрих Время, в котором меня нет vremya_v_kotorom_menya_net 5.0
Александр Перевезенцев Таёжные приключения tayozhnye_priklyucheniya 5.0
Татьяна Бердникова Перчатка Соломона perchatka_solomona 5.0
Елена Фёдорова Загадки Фауста zagadki_fausta 5.0
Вероника Ткачёва Сказки об Одуванчиках skazki_ob_oduvanchikakh 5.0
Николай Ростиславович Богатырёв Обратная сторона obratnaya_storona_1 4.9
Линара Юркина Танцы с шаманом tancy_sshamanom 5.0
Даниил Сергеевич Веселов Холодный оскал kholodnyi_oskal 5.0
Леонид Андронов Амулет amulet 5.0
150
Александр Шерман Падение Горизонта padenie_gorizonta 4.7
Сергей Курфюрстов Дом Кошкина: Маша Бланк dom_koshkina_masha_blank 5.0
Эндрю Огнёфф Хроники Древних Малефистериум khroniki_drevnikh_malefisterium_1 5.0
Руслан Калитин Баржизнь barzhizn 5.0
Софи Лейн Длинные тени старых грехов dlinnye_teni_starykh_grekhov 5.0
Алексей Челпаченко Козацкий шлях kozackii_s

Геннадий Логинов По ту сторону страницы po_tu_storonu_stranicy 5.0
Джанет Нортон На Другой стороне na_drugoi_storone 5.0
Екатерина Заостровцева Если ребенку трудно в школе esli_rebenku_trudno_v_shkole 4.0
Эдуард Шифман П у т ь Э р т о erto 5.0
Людмила Киндерская Таланты и преступники talanty_i_prestupniki 5.0
Гарри Ларин Назад в СССР с кучей баксов nazad_v_sssr_s_kuchei_baksov 5.0
Sergei Davidoff Тюрьмерика tyurmerika 4.9
Ислам Вагабонды vagabondy 5.0
Малика Атей Я никогда не ya_nikogda_ne 5.0
Дарья Новицкая Лекарство для сердца lekarstvo_dlya_serdca 4.7
Татьяна Володина Зверь в тебе zver_v_tebe 4.9
Алексей Иванов Еврей в хорошем смысле этого слова evrei_v_khoroshem_smysle_etogo_slova 5.0
Алекс Лоренц 1994 1994 5.0
Майя Еремеева Протекторы: Иллюзия выбора protektory_illyuziya_vybora 5.0
Ольга Сорокина По ту сторону любви po_tu_storonu_lyubvi 5.0
Тамасуриа Санти 33 жизни 33_zhizni 4.9
Андрей Окунь Забытая жизнь zabytaya_zhizn 5.0
Ольга Верещагина Нарекаю тебя женщиной narekayu_tebya_zhe

Анастасия Исаева Сценарий правится на ходу scenarii_pravitsya_na_khodu 5.0
Максим Бодягин Привратник privratnik 5.0
Ирина Александровна Агапова 12 моментов грусти 12_momentov_grusti 5.0
Виктория Романова Шлюха поневоле nakazanie_1 5.0
Нагима Харесова Тихая Америка tikhaya_amerika 5.0
Цзинь Цхон Ли Тайна татуировки taina_tatuirovki 5.0
Алиса Тишинова Букет из мать-и-мачехи, или Сказка для взрослых buket_iz_mat-i-machekhi_ili_skazka_dlya_vzroslykh 5.0
Юлия Тимур За чашечкой турецкого кофе za_chashechkoi_tureckogo_kofe 5.0
Алексей Шнейдер Мой бабаян moi_babayan 5.0
Татьяна Сальвони Она просто сказала ДА bludniza 4.7
Антон Бородачёв Всё, что я о ней помню vsyo_chto_ya_o_nei_pomnyu 4.8
Геннадий Мурзин Страсти людские strasti_lyudskie 4.9
Вячеслав Викторович Серёгин Память pamyat_6 5.0
425
Жюли Торш Давние счёты davnie_schyoty 5.0
Елизавета Благовест Человек и море & Пока ты была на море chelovek_i_more_and_poka_ty_byla_na_more 4.8
Джанет Нортон На Другой стороне na_drugoi_storone_1 4.8
Дмит

Ева Гончар Каникулы в «Сердце гор» kanikuly_v_serdce_gor 5.0
Александр Поехавший Завтра на РАБоту zavtra_na_rabotu 5.0
Мария Егорова Люди ночи lyudi_nochi 5.0
Максим Лист Нас — нет! nas_net 5.0
Сергей Дейкун Хранитель Разума hranitel_razuma 5.0
Константин Кривчиков Кукла для кандидата kukla_dlya_kandidata 5.0
Кристина Робер Преданные. Белое с кровью predannye_1 5.0
Эльдар Ахадов На семнадцати языках na_semnadcati_yazykakh 5.0
Серёжа Зубков Новогоднее одиночество novogodnee_odinochestvo 5.0
Светлана Гололобова Урод рода человеческого urod_roda_chelovecheskogo 5.0
gambaru КРИСМАСу krismasu 5.0
Инна Анатольевна Купреева Игры судьбы igry_sudby_1 5.0
Иван Варовин Разум вне Меня razum_vnemenya 4.4
Aria Tango Подарок podarok_4 4.3
Наталия Фелиз Что такое любовь... chto_takoe_lyubov_2 5.0
Наталия Махровская Важный свидетель vazhnyi_svidetel 5.0
Геннадий Логинов Долг палача dolg_palacha 5.0
Юлия Иосифовна Ершова Рождается весна rozhdaetsya_vesna 5.0
Максим Танаевский Дух Свободы. В поисках себя

...

Мерьем Йолач Открывающая пути otkryvayushaya_puti 5.0
Наталья Ружицкая и ещё 1 автор Дневник Осени dnevnik_oseni 5.0
Андрей Сергеевич Бесквитов Обет молчания obet_molchaniya 5.0
Сергей Борисович Протасов Крылья для одиночек krylya_dlya_odinochek 5.0
Natasha J. Osman Зови меня Кицунэ zovi_menya_kicune 5.0
Петр Тушнолобов Виктор Йог на прозе.ру viktor_iog_na_proze_ru 4.7
Екатерина Гладышева и ещё 1 автор По следам Белого Кролика po_sledam_belogo_krolika 5.0
Александр Петрашов Селлтирианд selltiriand 5.0
Акс Мэтт Семь дней будущего sem_dnei_budushego 5.0
Евгения Задорожная Его любовь ego_lyubov 5.0
Олег Бакиев Приключения кота Хеппи priklyucheniya_kota_kheppi 5.0
Мелисса Эйр Тайна Стеклянной Пещеры taina_steklyannoi_peshery 4.5
Людмила Викторовна Шилова Всё хорошо, или Шесть миллионов в пластилине vsyo_khorosho_ili_shest_millionov_v_plastiline 5.0
Елена Тимошенко-Седьмая Чужая постель chuzhaya_postel 5.0
А. С. Грехова Все так, как должно быть vse_tak_kak_dolzhno_byt 5.0
Иван Иван-Чаев Жиз

Алия Саят Тайна роз taina_roz 5.0
Иван Панкратов Бестеневая лампа bestenevaya_lampa 5.0
Илья Фролов Перешагнуть Россию pereshagnut_rossiyu 5.0
Михаил Громов Кто хорошо работает, кто пашет на Руси?.. kto_khorosho_rabotaet_kto_pashet_na_rusi 1.0
Элис де Салье Страсть и приличия strast_i_prilichiya 5.0
Наталья Щербатюк Юго-Восточная Азия yugo-vostochnaya_aziya 5.0
Ольга Клушина Чудесное пространство chudesnoe_prostranstvo 5.0
Дмитрий Бондаренко Да будет так da_budet_tak_1 5.0
Валентин Беляков Диффузия миров diffuziya_mirov 5.0
Кристина Новосельцева Кирро kirro 4.9
Марина Кирилловна Махортова Фортуна майора Колюбакина fortuna_maiora_kolyubakina 5.0
Даниил Александрович Шишкин Радуга судьбы raduga_sudby 5.0
Андрей Анатольевич Савкин Страшилки, кричалки, поучалки strashilki_krichalki_pouchalki 3.0
Михаил Леонидович Прядухин В тени больших вишневых деревьев v_teni_bolshikh_vishnevykh_derevev 5.0
Ольга Трушкова По лабиринту памяти po_labirinty_pamyati 5.0
Оксана Быстрицкая Зацепка за судьбу za

Светлана Фетисова Зеленое платье Надежды zelenoe_plate_nadezhdy 5.0
Анна Чуланова Девочка с изумрудной душой devochka_s_izumrudnoi_dushoi 5.0
Анатолий Семенович Власов Дневник одного путешествия dnevnik_odnogo_puteshestviya 5.0
Власов Алексей Северный Колосс. Предвестие Бури severnyi_koloss_predvestie_buri 5.0
Максим Мернес CryptoBoss cryptoboss 5.0
Антон Баков Государство – это ты! gosudarstvo_eto_ty 5.0
Ангел Света Мечта и вдохновение mechta_i_vdokhnovenie 4.7
Людмила Емелина и ещё 1 автор Волшебные приключения в мире финансов volshebnye_priklyucheniya_v_mire_finansov 5.0
Елена Чиркова Оставьте ангелов без работы sryvai_s_gvozdya_krylya 5.0
Елена Протас «Магазин снов» мистера Анимуса magazin_snov_mistera_animusa 5.0
Елена Рощина Знакомый незнакомец znakomyi_neznakomec 4.8
1675
Юлия Монакова Галинкина любовь galinkina_lyubov 5.0
Максим Евгеньевич Лебедев Бандитская любовь banditskaya_lyubov 5.0
Евгений Мороков Мыс Плака mys_plaka 4.7
Аарон Фан Сириус и тайны Нибиру sirius_i_tainy_nibi

Хелью Ребане Кот в лабиринте kot_v_labirinte 5.0
Шимон Гойзман Воспоминания незнаменитого vospominaniya_neznamenitogo_1 5.0
Василий Петрович Лабецкий Сады Ябоневни sady_yabonevni 5.0
Борис Штейман Ночная пуля nochnaya_pulya 5.0
Павел Васильевич Кузнецов Загадка уральской короны zagadka_uralskoi_korony 5.0
Аликс Ал Трапп trapp 5.0
Александра Ковальски Блики Солнца bliki_solnca 5.0
Наталия Мстительная Голубое утро goluboe_utro 5.0
Александр Гельманов Древо прошлой жизни drevo_proshloi_zhizni_1 5.0
Максим Крамар Вожделенный сыр vozhdelennyi_syr 5.0
Виктор Александрович Пушкарев За окном до сих пор идёт снег za_oknom_do_sikh_por_idyot_sneg 4.9
Михаил Геннадиевич Дерыведмидь Машина времени за ухом mashina_vremeni_za_ukhom 5.0
Гай Себеус Сказки о временах, когда кони были крылатыми 11_novykh_skifskikh_skazok 5.0
Денис Бобкин Мазь Якова Брюса maz_yakova_bryusa 5.0
Натали Анжети Полтора килограмма poltora_kilogramma 5.0
Вера Маленькая Однажды в Коктебеле odnazhdy_v_koktebele 5.0
Борис Жук Испо

Анко Джэер Самый человечный цвет samyi_chelovechnyi_cvet 5.0
Антон Краянский Пришелец с планеты Земля prishelec_s_planety_zemlya 5.0
Анжела Конн Девственник devstvennik 5.0
Игорь Исаев Наброски души nabroskidushi 2.0
Лена Терехова Из жизни женщины… iz_zhizni_zhenshiny 4.5
1950
Елисей Шабельников Завтраки у Мротских zavtraki_u_mrotskikh 3.4
Наталья Патрацкая Мудрец Изумрудного округа mudrec_izumrudnogo_okruga 5.0
Вера Мосова У изголовья прошлых лет u_izgolovya_proshlykh_let 5.0
Шамиль Магомедович Арсланов Дом у озера dom_u_ozera 5.0
Ксения Семенихина и ещё 9 авторов Шальные истории shalnye_istorii 4.2
Evgenii Shan Путь в Шамбалу put_v_shambalu 5.0
Евгений Гущин Записки серфера zapiski_serfera 5.0
Неточка Незванова Мгновение двух жизней mgnovenie_dvukh_zhiznei 5.0
Илона Якимова Белокурый. Права наследства belokuryi_prava_nasledstva 5.0
Александр Владимирович Соколов Эпоха покаяния ehpoha_pokayaniya 4.7
Эмануил Бланк Реванш revansh_1 5.0
Павел Волчик Плато Ветров plato_vetrov 5.0
Алексей 

In [5]:
import pandas as pd
char_range = []

for filename in sorted(os.listdir(dir_ridero)):
    with open(os.path.join(dir_ridero, filename), 'r') as book:
        content = book.read()

    content = content.replace('?..', '?').replace('!..', '!').replace('...', '…').replace('--', '–')
    content = re.sub('[…]+', ' … ', content)
    content = re.sub('[_]+', '_', content)
    content = re.sub('[\n]+', ' \n ', content)
    content = re.sub('[*]+', '*', content)
    content = re.sub('\xa0', ' ', content)
    with open(os.path.join(dir_ridero, filename), 'w') as book:
        book.write(content)
        size = len(content)
        if size < 10:
            print(filename)
    char_range.append(size)
        
char_range = pd.Series(char_range)

In [81]:
char_range.min()

167

In [80]:
sorted(os.listdir(dir_ridero))[191]

'mat_kakoi_byt_nelzya'

In [49]:
d = pd.DataFrame()
d['id'] = range(10)
d['target'] = range(10)
d['group'] = range(10)

In [53]:
d.groupby('group').apply(lambda x: x.sort_values('target', ascending=False).index[0])

group
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64